In [2]:
import requests
from bs4 import BeautifulSoup
import os

# 创建用于存储数据的文件夹
if not os.path.exists('Premier_League_data'):
    os.makedirs('Premier_League_data')

# 设置需要爬取的赛季年份
seasons = range(2000, 2023)

# 遍历每个赛季
for season in seasons:
    # 构造数据网址
    url = f'https://www.football-data.co.uk/mmz4281/{str(season)[2:]}{str(season+1)[2:]}/E0.csv'
    # 发送 HTTP 请求并获取响应
    response = requests.get(url)
    # 使用 BeautifulSoup 解析 HTML 页面
    soup = BeautifulSoup(response.content, 'html.parser')
    # 从页面中获取数据内容
    data = soup.get_text()
    # 构造新文件名
    filename = f'Premier_League_data/{season}-{season+1}.csv'
    # 将数据写入新文件中
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(data)

In [ ]:
pip install python-docx

In [1]:
import requests
import docx
import os

# 创建用于存储数据的文件夹
if not os.path.exists('Premier_League_data'):
    os.makedirs('Premier_League_data')

# 发送 HTTP 请求并获取响应
response = requests.get('https://www.football-data.co.uk/notes.txt')

# 将响应内容转换为字符串   
content = response.text

# 创建 Word 文档对象
document = docx.Document()

# 将每行文本添加到 Word 文档中
for line in content.split('\n'):
    document.add_paragraph(line)

# 将 Word 文档保存为文件
filename = 'Premier_League_data/notes.docx'
document.save(filename)


In [2]:
import os
import pandas as pd

# 创建用于存储数据的文件夹
data_folder = 'Premier_League_data'

# 设置需要统计的赛季年份
seasons = range(2000, 2023)

# 初始化一个列表用于存储每个赛季的 DataFrame
season_dataframes = []

# 遍历每个赛季
for season in seasons:
    # 构造文件名
    filename = f'{data_folder}/{season}-{season+1}.csv'
    
    # 读取 CSV 文件
    df = pd.read_csv(filename, error_bad_lines=False)

    # 计算每支球队的主场和客场积分
    home_points = df.groupby('HomeTeam')['FTR'].apply(lambda x: (x == 'H').sum() * 3 + (x == 'D').sum())
    away_points = df.groupby('AwayTeam')['FTR'].apply(lambda x: (x == 'A').sum() * 3 + (x == 'D').sum())

    # 合并主场和客场积分
    total_points = home_points.add(away_points, fill_value=0)
    
    # 对球队积分进行排序
    sorted_points = total_points.sort_values(ascending=False)
    
    # 将排名和赛季信息添加到 DataFrame 中
    season_df = sorted_points.reset_index().rename(columns={'index': 'Team', 0: 'Points'})
    season_df['Rank'] = season_df.index + 1
    season_df['Season'] = f'{season}-{season+1}'
    
    # 添加到赛季 DataFrame 列表中
    season_dataframes.append(season_df)

# 合并所有赛季的 DataFrame
combined_df = pd.concat(season_dataframes, ignore_index=True)

# 将结果保存为 source.csv 文件
combined_df.to_csv('Premier_League_data/source.csv', index=False)


/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/223687487.py:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(filename, error_bad_lines=False)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/223687487.py:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(filename, error_bad_lines=False)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/223687487.py:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(filename, error_bad_lines=False)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/223687487.py:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version

In [3]:
import os
import pandas as pd

# 创建用于存储数据的文件夹
data_folder = 'Premier_League_data'

# 设置需要统计的赛季年份
seasons = range(2000, 2023)

# 初始化一个列表用于存储每个赛季的 DataFrame
season_dataframes = []

# 遍历每个赛季
for season in seasons:
    # 构造文件名
    filename = f'{data_folder}/{season}-{season+1}.csv'
    
    # 读取 CSV 文件
    df = pd.read_csv(filename, error_bad_lines=False)

    # 计算每支球队的主场和客场数据
    home_stats = df.groupby('HomeTeam').agg(
        {'FTR': [('HomeWins', lambda x: (x == 'H').sum()),
                 ('HomeDraws', lambda x: (x == 'D').sum()),
                 ('HomeLosses', lambda x: (x == 'A').sum())],
         'FTHG': [('HomeGoals', 'sum')],
         'FTAG': [('HomeConceded', 'sum')],
         'HS': [('HomeShots', 'sum')],
         'AS': [('HomeShotsAgainst', 'sum')],
         'HF': [('HomeFouls', 'sum')],
         'AF': [('HomeFoulsAgainst', 'sum')],
         'HY': [('HomeYellowCards', 'sum')],
         'AY': [('HomeYellowCardsAgainst', 'sum')],
         'HR': [('HomeRedCards', 'sum')],
         'AR': [('HomeRedCardsAgainst', 'sum')],
         'HC': [('HomeCorners', 'sum')],
         'AC': [('HomeCornersAgainst', 'sum')]}
    )

    away_stats = df.groupby('AwayTeam').agg(
        {'FTR': [('AwayWins', lambda x: (x == 'A').sum()),
                 ('AwayDraws', lambda x: (x == 'D').sum()),
                 ('AwayLosses', lambda x: (x == 'H').sum())],
         'FTAG': [('AwayGoals', 'sum')],
         'FTHG': [('AwayConceded', 'sum')],
         'AS': [('AwayShots', 'sum')],
         'HS': [('AwayShotsAgainst', 'sum')],
         'AF': [('AwayFouls', 'sum')],
         'HF': [('AwayFoulsAgainst', 'sum')],
         'AY': [('AwayYellowCards', 'sum')],
         'HY': [('AwayYellowCardsAgainst', 'sum')],
         'AR': [('AwayRedCards', 'sum')],
         'HR': [('AwayRedCardsAgainst', 'sum')],
         'AC': [('AwayCorners', 'sum')],
         'HC': [('AwayCornersAgainst', 'sum')]}
    )

    # 重置索引并合并主场和客场数据
    home_stats.reset_index(inplace=True)
    away_stats.reset_index(inplace=True)
    total_stats = pd.merge(home_stats, away_stats, left_on='HomeTeam', right_on='AwayTeam')

    # 删除重复的球队名列，并添加赛季信息
    total_stats = total_stats.drop(columns=['AwayTeam'])
    total_stats = total_stats.rename(columns={'HomeTeam': 'Team'})
    total_stats['Season'] = f'{season}-{season+1}'
    season_dataframes.append(total_stats)

combined_df = pd.concat(season_dataframes, ignore_index=True)
combined_df.columns = ['Team', 'HomeWins', 'HomeDraws', 'HomeLosses', 'HomeGoals', 'HomeConceded', 'HomeShots', 'HomeShotsAgainst', 'HomeFouls', 'HomeFoulsAgainst', 'HomeYellowCards', 'HomeYellowCardsAgainst', 'HomeRedCards', 'HomeRedCardsAgainst', 'HomeCorners', 'HomeCornersAgainst', 'AwayWins', 'AwayDraws', 'AwayLosses', 'AwayGoals', 'AwayConceded', 'AwayShots', 'AwayShotsAgainst', 'AwayFouls', 'AwayFoulsAgainst', 'AwayYellowCards', 'AwayYellowCardsAgainst', 'AwayRedCards', 'AwayRedCardsAgainst', 'AwayCorners', 'AwayCornersAgainst', 'Season']
combined_df['Wins'] = combined_df['HomeWins'] + combined_df['AwayWins']
combined_df['Draws'] = combined_df['HomeDraws'] + combined_df['AwayDraws']
combined_df['Losses'] = combined_df['HomeLosses'] + combined_df['AwayLosses']
combined_df['Goals'] = combined_df['HomeGoals'] + combined_df['AwayGoals']
combined_df['Conceded'] = combined_df['HomeConceded'] + combined_df['AwayConceded']
combined_df['Shots'] = combined_df['HomeShots'] + combined_df['AwayShots']
combined_df['ShotsAgainst'] = combined_df['HomeShotsAgainst'] + combined_df['AwayShotsAgainst']
combined_df['Fouls'] = combined_df['HomeFouls'] + combined_df['AwayFouls']
combined_df['FoulsAgainst'] = combined_df['HomeFoulsAgainst'] + combined_df['AwayFoulsAgainst']
combined_df['YellowCards'] = combined_df['HomeYellowCards'] + combined_df['AwayYellowCards']
combined_df['YellowCardsAgainst'] = combined_df['HomeYellowCardsAgainst'] + combined_df['AwayYellowCardsAgainst']
combined_df['RedCards'] = combined_df['HomeRedCards'] + combined_df['AwayRedCards']
combined_df['RedCardsAgainst'] = combined_df['HomeRedCardsAgainst'] + combined_df['AwayRedCardsAgainst']
combined_df['Corners'] = combined_df['HomeCorners'] + combined_df['AwayCorners']
combined_df['CornersAgainst'] = combined_df['HomeCornersAgainst'] + combined_df['AwayCornersAgainst']
combined_df['Points'] = combined_df['Wins'] * 3 + combined_df['Draws']
combined_df.to_csv('Premier_League_data/total.csv', index=False)

/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2950260104.py:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(filename, error_bad_lines=False)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2950260104.py:64: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  total_stats = total_stats.drop(columns=['AwayTeam'])
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2950260104.py:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(filename, error_bad_lines=False)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2950260104.py:64: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  total_stats =

/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2950260104.py:64: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  total_stats = total_stats.drop(columns=['AwayTeam'])
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2950260104.py:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(filename, error_bad_lines=False)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2950260104.py:64: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  total_stats = total_stats.drop(columns=['AwayTeam'])
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2950260104.py:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.re

In [4]:
import pandas as pd

# 读取 total.csv 文件
total_df = pd.read_csv('Premier_League_data/total.csv')

# 更改列顺序
cols = total_df.columns.tolist()
cols.insert(1, cols.pop(cols.index('Season')))
cols.insert(2, cols.pop(cols.index('Points')))
total_df = total_df[cols]

# 获取所有赛季
seasons = total_df['Season'].unique()

# 初始化一个空的 DataFrame 用于存储带有排名的数据
ranked_df = pd.DataFrame()

# 遍历每个赛季
for season in seasons:
    # 获取该赛季的数据
    season_df = total_df[total_df['Season'] == season]

    # 按照积分、净胜球、进球数对球队进行排序
    season_df = season_df.sort_values(by=['Points', 'Goals', 'Conceded'], ascending=[False, False, True])

    # 生成排名
    season_df['Rank'] = range(1, len(season_df) + 1)

    # 将排名插入到 DataFrame 中
    ranked_df = ranked_df.append(season_df)

# 将排名列移到第四列
cols = ranked_df.columns.tolist()
cols.insert(3, cols.pop(cols.index('Rank')))
ranked_df = ranked_df[cols]

# 将带有排名的数据保存到新的 CSV 文件中
ranked_df.to_csv('Premier_League_data/total_with_rank.csv', index=False)


/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/444306875.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_df = ranked_df.append(season_df)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/444306875.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_df = ranked_df.append(season_df)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/444306875.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_df = ranked_df.append(season_df)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/444306875.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_df = rank

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 设置请求头，伪装成浏览器
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.51'}

# 请求 Sky Sports 的 Premier League 比赛页面
url = "https://www.skysports.com/premier-league-fixtures"
response = requests.get(url, headers=headers)

# 确保请求成功
response.raise_for_status()

# 解析网页源代码
soup = BeautifulSoup(response.content, 'html.parser')

# 获取比赛数据
matches = soup.select('.fixres__item')
match_data = []

for match in matches:
    date = match.select_one('.matches__date').text.strip()
    home_team = match.select_one('.matches__participant--side1 .swap-text__target').text.strip()
    away_team = match.select_one('.matches__participant--side2 .swap-text__target').text.strip()

    match_data.append({
        'Date': date,
        'Home Team': home_team,
        'Away Team': away_team
    })

# 保存数据到CSV文件
remaining_matches_df = pd.DataFrame(match_data)
remaining_matches_df.to_csv('Premier_League_data/remaining_matches.csv', index=False)


In [5]:
import pandas as pd
import os

# 创建一个空的DataFrame，用于存储所有赛季的数据
all_seasons_df = pd.DataFrame()

# 设置需要整合的赛季年份
seasons = range(2000, 2023)

# 遍历每个赛季
for season in seasons:
    # 构造文件名
    filename = f'Premier_League_data/{season}-{season+1}.csv'
    
    # 确保文件存在
    if os.path.exists(filename):
        # 读取赛季数据到一个临时DataFrame，忽略解析错误的行
        temp_df = pd.read_csv(filename, error_bad_lines=False)
        
        # 添加赛季列
        temp_df['Season'] = f'{season}-{season+1}'
        
        # 将临时DataFrame添加到所有赛季的DataFrame中
        all_seasons_df = all_seasons_df.append(temp_df, ignore_index=True)

# 将所有赛季的数据保存到一个CSV文件中
all_seasons_df.to_csv('Premier_League_data/all_seasons_data.csv', index=False)


/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2044910193.py:18: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  temp_df = pd.read_csv(filename, error_bad_lines=False)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2044910193.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_seasons_df = all_seasons_df.append(temp_df, ignore_index=True)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2044910193.py:18: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  temp_df = pd.read_csv(filename, error_bad_lines=False)
/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/2044910193.py:24: FutureWarning: The frame.append method is deprecated and will be removed from 

In [6]:
import pandas as pd

# 读取 all_seasons_data.csv 文件
all_seasons_df = pd.read_csv("Premier_League_data/all_seasons_data.csv")

# 保留以下列名
relevant_columns = [
    'Date', 'Season', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 
    'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 
    'HF', 'AF', 'HY', 'AY', 'HR', 'AR'
]

# 仅保留与比赛结果和统计数据相关的列
all_seasons_df = all_seasons_df[relevant_columns]

# 将筛选后的数据保存到一个新的 CSV 文件中
all_seasons_df.to_csv("Premier_League_data/all_seasons_data_relevant.csv", index=False)


/var/folders/qp/w4l86_z57cg81831df6lkm6w0000gn/T/ipykernel_1434/978793329.py:4: DtypeWarning: Columns (109) have mixed types. Specify dtype option on import or set low_memory=False.
  all_seasons_df = pd.read_csv("Premier_League_data/all_seasons_data.csv")


In [7]:
import pandas as pd

# 读取 all_seasons_data.csv 文件
all_seasons_df = pd.read_csv("Premier_League_data/all_seasons_data_relevant.csv")

# 获取所有参加过英超联赛的球队名称
prem_teams = pd.concat([all_seasons_df['HomeTeam'], all_seasons_df['AwayTeam']]).unique()

# 打印球队列表
print(prem_teams)


['Charlton' 'Chelsea' 'Coventry' 'Derby' 'Leeds' 'Leicester' 'Liverpool'
 'Sunderland' 'Tottenham' 'Man United' 'Arsenal' 'Bradford' 'Ipswich'
 'Middlesbrough' 'Everton' 'Man City' 'Newcastle' 'Southampton' 'West Ham'
 'Aston Villa' 'Bolton' 'Blackburn' 'Fulham' 'Birmingham' 'West Brom'
 'Portsmouth' 'Wolves' 'Norwich' 'Crystal Palace' 'Wigan' 'Reading'
 'Sheffield United' 'Watford' 'Hull' 'Stoke' 'Burnley' 'Blackpool' 'QPR'
 'Swansea' 'Cardiff' nan 'Bournemouth' 'Brighton' 'Huddersfield'
 'Brentford' "Nott'm Forest"]


In [3]:
import pandas as pd

# 读取 remaining_matches.csv 文件
remaining_matches_df = pd.read_csv("Premier_League_data/remaining_matches.csv")

# 获取所有参赛球队名称
teams = pd.concat([remaining_matches_df['Home Team'], remaining_matches_df['Away Team']]).unique()

# 打印球队列表
print(teams)


['Bournemouth' 'Leeds' 'Leicester' 'Chelsea' 'Man United' 'West Ham'
 'Aston Villa' 'Brentford' 'Fulham' 'Tottenham' 'Wolves' 'Southampton'
 'Liverpool' 'Everton' 'Man City' "Nott'm Forest" 'Arsenal'
 'Crystal Palace' 'Newcastle' 'Brighton']


In [9]:
import pandas as pd

team_name_mapping = {
    'West Ham United': 'West Ham',
    'Newcastle United': 'Newcastle',
    'Everton': 'Everton',
    'Bournemouth': 'Bournemouth',
    'Leeds United': 'Leeds',
    'Leicester City': 'Leicester',
    'Chelsea': 'Chelsea',
    'Manchester United': 'Man United',
    'Aston Villa': 'Aston Villa',
    'Brentford': 'Brentford',
    'Fulham': 'Fulham',
    'Tottenham Hotspur': 'Tottenham',
    'Wolverhampton Wanderers': 'Wolves',
    'Southampton': 'Southampton',
    'Liverpool': 'Liverpool',
    'Manchester City': 'Man City',
    'Nottingham Forest': "Nott'm Forest",
    'Arsenal': 'Arsenal',
    'Crystal Palace': 'Crystal Palace',
    'Brighton and Hove Albion': 'Brighton'
}

# 读取 remaining_matches.csv 文件
remaining_matches_df = pd.read_csv("Premier_League_data/remaining_matches.csv")

# 使用字典替换球队名称
remaining_matches_df['Home Team'] = remaining_matches_df['Home Team'].replace(team_name_mapping)
remaining_matches_df['Away Team'] = remaining_matches_df['Away Team'].replace(team_name_mapping)

# 保存更新后的数据到 CSV 文件
remaining_matches_df.to_csv("Premier_League_data/remaining_matches_updated.csv", index=False)
